In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
from keras.models import load_model

%matplotlib inline

Using TensorFlow backend.


In [2]:
%load_ext autoreload 
%autoreload 1 
%aimport inception_resnet_v1

model_path = './model/keras/model/facenet_keras.h5' 
weights_path='./model/keras/weight/facenet_keras_weights.h5' 
model = inception_resnet_v1.InceptionResNetV1(input_shape=(160, 160, 3), classes=128, dropout_keep_prob=0.8, weights_path=weights_path)

#model = load_model(model_path)

In [32]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
__________________________________________________________________________________________________
Conv2d_2a_

In [33]:
X_train = np.load('./input/X_train_160.npy')
Y_train = np.load('./input/Y_train.npy')
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train,Y_train,test_size=0.2, random_state=42)

In [4]:
celeberty_names = np.load('./input/names.npy')
file_names = np.load('./input/test_files.npy')

In [5]:
def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [13]:
def calc_embs(aligned_images, batch_size=32):
    pd = []
    pd.append(model.predict(aligned_images,batch_size = batch_size, verbose = 1))
    embs = l2_normalize(np.concatenate(pd))

    return embs

In [34]:
embs_train = calc_embs(X_train)    
embs_valid = calc_embs(X_valid)

1043/1043 [==============================] - 8s 8ms/step


### SVM

In [42]:
clf = SVC(kernel='linear', probability=True,).fit(embs_train, np.argmax(Y_train,axis=1))
clf.score(embs_valid,np.argmax(Y_valid,axis=1))

0.96452540747842763

## submission

In [44]:
X_test = np.load('./input/X_test_160.npy')
embs_test = calc_embs(X_test)
pred = clf.predict(embs_test)

2628/2628 [==============================] - 20s 8ms/step


In [48]:
import pandas as pd
names = celeberty_names[pred]
names = pd.Series(names,name="celebrity_name")
submission = pd.concat([pd.Series(file_names,name = "image_label"),names],axis = 1)

submission.to_csv("celeberty_prediction.csv",index=False)